Establish connection

In [1]:
%reload_ext google.cloud.bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:\\Users\\Cermati\\AppData\\Roaming\\gcloud\\application_default_credentials.json"
print(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

C:\Users\Cermati\AppData\Roaming\gcloud\application_default_credentials.json


Insert query here

In [2]:
%%bigquery df --project athena-179008
With data_disbursement as (
    SELECT 
        mdr.purchase_transaction_id
        , CAST(SUM(CASE WHEN mdl.type='DISBURSEMENT' AND mdl.status='DONE' THEN mdl.amount ELSE 0 END) AS FLOAT64) as disbursement
        , CAST(SUM(CASE WHEN mdl.type='MERCHANT_DISCOUNT_RATE' AND mdl.status='DONE' THEN mdl.amount ELSE 0 END) AS FLOAT64) as mdr
        , CAST(SUM(CASE WHEN mdl.type='DISBURSEMENT' AND mdl.status='PENDING' THEN mdl.amount ELSE 0 END) AS FLOAT64) as disbursement_pending
        , CAST(SUM(CASE WHEN mdl.type='MERCHANT_DISCOUNT_RATE' AND mdl.status='PENDING' THEN mdl.amount ELSE 0 END) AS FLOAT64) as mdr_pending
    FROM data-platform-indodana.vayu.indodana_merchant_purchase_transactions pt
    INNER JOIN data-platform-indodana.vayu.indodana_merchant_merchant_disbursement_requests mdr ON mdr.purchase_transaction_id=pt.id
    INNER JOIN data-platform-indodana.vayu.indodana_merchant_merchant_disbursement_ledgers mdl ON mdl.id=mdr.merchant_disbursement_ledger_id
    GROUP BY 1
)
, data_refund as (
    SELECT 
        mrr.purchase_transaction_id
        , CAST(SUM(CASE WHEN mdl.type='REFUND' AND mdl.status='DONE' THEN mdl.amount ELSE 0 END) AS FLOAT64) as refund
        , CAST(SUM(CASE WHEN mdl.type='REFUND_MERCHANT_DISCOUNT_RATE' AND mdl.status='DONE' THEN mdl.amount ELSE 0 END) AS FLOAT64) as refund_mdr
        , CAST(SUM(CASE WHEN mdl.type='REFUND' AND mdl.status='PENDING' THEN mdl.amount ELSE 0 END) AS FLOAT64) as refund_pending
        , CAST(SUM(CASE WHEN mdl.type='REFUND_MERCHANT_DISCOUNT_RATE' AND mdl.status='PENDING' THEN mdl.amount ELSE 0 END) AS FLOAT64) as refund_mdr_pending
        , MAX(mdl.created_at) as refund_date
        , MAX(mdl.disburse_date) as refund_date1
    FROM data-platform-indodana.vayu.indodana_merchant_purchase_transactions pt
    INNER JOIN data-platform-indodana.vayu.indodana_merchant_merchant_refund_requests mrr ON mrr.purchase_transaction_id=pt.id
    INNER JOIN data-platform-indodana.vayu.indodana_merchant_merchant_disbursement_ledgers mdl ON mdl.id=mrr.merchant_disbursement_ledger_id
    GROUP BY 1
)

SELECT 
    c.entity_id
    , c.status
    , c.purchase_status
    , c.type
    , c.approved_date
    , c.loan_amount
    , c.purchase_amount
    , d.* except(purchase_transaction_id)
    , r.* except(purchase_transaction_id)
FROM athena-179008.vayu_data_mart.view_trx_contracts c 
LEFT JOIN data_disbursement d ON d.purchase_transaction_id=c.purchase_transaction_id
LEFT JOIN data_refund r ON r.purchase_transaction_id=c.purchase_transaction_id

Olah Data

In [3]:
df.describe()

,loan_amount,purchase_amount,disbursement,mdr,disbursement_pending,mdr_pending,refund,refund_mdr,refund_pending,refund_mdr_pending
count,9.157030e+05,9.157030e+05,9.157030e+05,915703.000000,9.157030e+05,915703.000000,5.021200e+04,50212.000000,5.021200e+04,50212.000000
mean,2.518684e+05,2.550713e+05,2.529277e+05,-1891.782555,1.394744e+03,-9.043976,-3.349475e+05,1545.649886,-4.695036e+03,24.094687
std,4.588692e+05,4.640203e+05,4.604032e+05,3831.175907,4.227333e+04,284.188653,6.480914e+05,3894.609651,9.120254e+04,484.644205
min,0.000000e+00,1.000000e+03,0.000000e+00,-87905.000000,0.000000e+00,-56722.210000,-8.000000e+06,0.000000,-7.200000e+06,0.000000
25%,2.022500e+04,2.041850e+04,2.018500e+04,-1912.500000,0.000000e+00,0.000000,-2.808000e+05,0.000000,0.000000e+00,0.000000
50%,9.925900e+04,1.000000e+05,9.900000e+04,-550.000000,0.000000e+00,0.000000,-7.295550e+04,0.000000,0.000000e+00,0.000000
75%,2.560310e+05,2.580000e+05,2.560800e+05,0.000000,0.000000e+00,0.000000,-1.500000e+04,1162.920000,0.000000e+00,0.000000
max,8.290204e+06,8.290204e+06,8.124400e+06,0.000000,7.200000e+06,0.000000,0.000000e+00,50597.900000,0.000000e+00,41704.990000


In [4]:
df.head()

,entity_id,status,purchase_status,type,approved_date,loan_amount,purchase_amount,disbursement,mdr,disbursement_pending,mdr_pending,refund,refund_mdr,refund_pending,refund_mdr_pending,refund_date,refund_date1
0,TRX-210LHL,FINISHED,PROCESSED,PRODUCT_INSTALLMENT,2020-12-24 04:22:30.794086+00:00,86913,86913,86913.0,-869.13,0.0,0.0,NaN,NaN,NaN,NaN,NaT,NaT
1,TRX-H1UEUW,FINISHED,PROCESSED,PRODUCT_INSTALLMENT,2020-11-18 05:07:58.584476+00:00,64882,64882,64882.0,-648.82,0.0,0.0,NaN,NaN,NaN,NaN,NaT,NaT
2,TRX-T2SY7C,ACTIVE,PROCESSED,PRODUCT_INSTALLMENT,2021-04-18 04:47:33.987149+00:00,127000,127000,127000.0,-1295.40,0.0,0.0,NaN,NaN,NaN,NaN,NaT,NaT
3,TRX-D0MR2V,ACTIVE,PROCESSED,PRODUCT_INSTALLMENT,2020-12-28 17:05:14.305402+00:00,235802,235802,235802.0,-2358.02,0.0,0.0,NaN,NaN,NaN,NaN,NaT,NaT
4,TRX-ECEMM8,ACTIVE,PROCESSED,PRODUCT_INSTALLMENT,2021-04-16 01:32:42.538982+00:00,502806,502806,492750.0,0.00,0.0,0.0,NaN,NaN,NaN,NaN,NaT,NaT
